In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.nn import MSELoss, Linear, ReLU, Dropout

from joblib import dump, load
from Environment import calculate_val_metrics, save_metrics_to_txt, set_seed, Initlize
from Environment import MyDataset, MT_DNN, validation_MT_DNN
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
Metric_path = './utils/Regression_multi_task/'
scaler_x = load(Metric_path + 'Regression_multitask_x_scaler.joblib')
scaler_y = load(Metric_path + 'Regression_multitask_y_scaler.joblib')


file_path = './dataset/data_pre/Result_data_pre_include_fingerprints/'
x_pre = pd.read_csv(file_path + '07_Concatenated_data.csv')
x_pre = x_pre.iloc[:,1:]
x_pre = scaler_x.transform(x_pre)
X_pre = torch.tensor(x_pre, dtype=torch.float32)

y_pre = pd.read_csv('./dataset/data_pre/DATA_pre_use_to_predict_142.csv')
y_pre = y_pre.iloc[:,4:10]
y_pre = scaler_y.transform(y_pre)
Y_pre = torch.tensor(y_pre, dtype=torch.float32)

print(X_pre.shape)
print(Y_pre.shape)

pre_set = MyDataset(X_pre, Y_pre)
pre_loader = DataLoader(pre_set, batch_size=32, shuffle=False)

torch.Size([142, 2076])
torch.Size([142, 6])


In [3]:
set_seed(seed=42)
model = MT_DNN().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_function = MSELoss()
Initlize(model, init_type = 'kaiming_normal')
model.load_state_dict(torch.load(Metric_path + 'Regression_MT_DNN_best_model.pt'))

Y_pred_pre, Y_origin_pre, mask_pre, loss_pre = validation_MT_DNN(model, pre_loader, DEVICE)

Y_origin_pre = scaler_y.inverse_transform(Y_origin_pre)
Y_pred_pre = scaler_y.inverse_transform(Y_pred_pre)

output_path = './Result/regressor/DNN/'
np.savetxt(output_path + 'Y_pred_MT_DNN.csv', Y_pred_pre, delimiter=',')
np.savetxt(output_path + 'Y_origin_pre.csv', Y_origin_pre, delimiter=',')

pre_metrics = calculate_val_metrics(Y_pred_pre, Y_origin_pre, mask_pre)
for key, value in pre_metrics.items():
    print(f"The test set of {key}: {value:.4f}")

save_metrics_to_txt(pre_metrics, output_path + 'pre_metrics_MT_DNN.txt')

F:\_Fluorescence\GitHub\Code_NIRFluor\Environment.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(self.features[idx,:])


validation finished!
validation_loss: 0.2249
The test set of RMSE_0: 35.7422
The test set of RMSE_1: 33.6222
The test set of RMSE_2: 22.3825
The test set of RMSE_3: 0.2406
The test set of RMSE_4: 0.1798
